In [16]:
!pip install gradio


In [17]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import json

In [18]:
load_dotenv(override=True)
openai = OpenAI()

In [19]:
reader=PdfReader("resume.pdf")
resume=""
for page in reader.pages:
    text=page.extract_text()
    if text:
        resume+=text

In [20]:
print(resume)

+919021001351
hrishkul@gmail.com HrishikeshKulkarni
linkedin.com/in/hrishkul
github.com/hrishkul
EDUCATION
BachelorofScience,ComputerScience, VishwakarmaUniversity,CGPA:8.4 Aug2023—May2026
PROJECTS
MNISTNeuralNetwork, Jan2025—Feb2025
• DevelopedaneuralnetworkfromscratchinPythontoclassifyhandwrittendigitsusingtheMNISTdataset,achievingupto84.5%
testaccuracy.
• Implementedallcomponentsmanually,includingforwardpropagation,ReLUandsoftmaxactivations,andbackpropagationwith
stochasticgradientdescent.
• Designedacustomizablearchitecturewitha784-neuroninputlayer,hiddenlayers,anda10-neuronoutputlayertorepresentdigits
0–9.
• Ensuredtrainingon60,000imagesandevaluatedperformanceon10,000testimages,withaccuracyoutputsandpredictions
visualizedpost-training.
• AvoidedexternalframeworkslikeTensorFloworPyTorchtodeeplyunderstandthecoremechanicsofneuralnetworks.
• Coderepository: github.com/hrishkul/MNIST-neural-network
ProductRecommendationSystemusingInstacartDataset, June2025—Present
• Activelyworkingonde

In [21]:
with open("summary.txt", "r") as f:
    summary=f.read()
    

In [22]:
name="Hrishikesh Kulkarni"

In [23]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary} and \n\n## Resume:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [24]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback:str

In [25]:
evaluator_system_prompt = "You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n"
evaluator_system_prompt += f'''With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback.Please respond in the following JSON format:
{{"is_acceptable": true, "feedback": "..."}}
Only output valid JSON.'''


In [26]:
def convert_history(history):
    messages = []
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    return messages

def format_history(history):
    return "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in history])

def evaluator_user_prompt(reply, message, formatted_history):
    user_prompt = f"Here's the conversation between the User and the Agent:\n{formatted_history}\n\n"
    user_prompt += f"Here's the latest question asked by the User: {message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: {reply}\n\n"
    return user_prompt


In [27]:
def evaluate(reply, message, history) -> Evaluation:
    history_string = format_history(history)  # just formatting
    user_prompt = evaluator_user_prompt(reply, message, history_string)
    messages = [{"role": "system", "content": evaluator_system_prompt},{"role": "user", "content": user_prompt}]
    response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
    response_content = response.choices[0].message.content
    try:
        data = json.loads(response_content)
        return Evaluation(**data)
    except Exception as e:
        raise ValueError(f"Failed to parse evaluation response: {response_content}") from e


In [28]:
def rerun(reply,message,history,feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}]+  history +[{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [29]:
def chat(message, history):
    history=convert_history(history)
    system=system_prompt
    messages=[{"role":"system", "content": system}]+ history+ [{"role":"user", "content":message}]
    response=openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply=response.choices[0].message.content

    evaluation=evaluate(reply, message, history)
    if evaluation.is_acceptable:
        print("Evaluation: Acceptable")
    else:
        print("Evaluation: Rejected")
        print(evaluation.feedback)
        reply=rerun(reply,message,history,evaluation.feedback)
    return reply

In [ ]:
gr.ChatInterface(chat).launch()

/Users/hrishi/Desktop/codes/projects/agents/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Evaluation: Acceptable
Evaluation: Acceptable
Evaluation: Acceptable
Evaluation: Acceptable
